In [1]:
BRANCH = 'main'
!python -m pip install git+https://github.com/NVIDIA/NeMo.git@$BRANCH#egg=nemo_toolkit[nlp]

! pip install ipywidgets
! jupyter nbextension enable --py widgetsnbextension

In [2]:
from nemo.collections import nlp as nemo_nlp
from nemo.utils.exp_manager import exp_manager

import os
import wget 
import torch
import pytorch_lightning as pl
from omegaconf import OmegaConf

[NeMo W 2023-01-28 22:27:00 optimizers:55] Apex was not found. Using the lamb or fused_adam optimizer will error out.
2023-01-28 22:27:00.884118: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-01-28 22:27:01.434339: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-01-28 22:27:01.434385: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory
2023-01-28 22:27:01.434389: W tensorflow/compile

# Task Description
**Sentiment Analysis** is the task of detecting the sentiment in text. We model this problem as a simple form of a text classification problem. For example `Gollum's performance is incredible!` has a positive sentiment while `It's neither as romantic nor as thrilling as it should be.` has a negative sentiment.
.

# Dataset

In this tutorial we going to use [The Stanford Sentiment Treebank (SST-2)](https://nlp.stanford.edu/sentiment/index.html) corpus for sentiment analysis. This version of the dataset contains a collection of sentences with binary labels of positive and negative. It is a standard benchmark for sentence classification and is part of the GLUE Benchmark: https://gluebenchmark.com/tasks. Please download and unzip the SST-2 dataset from GLUE. It should contain three files of train.tsv, dev.tsv, and test.tsv which can be used for training, validation, and test respectively.





# NeMo Text Classification Data Format

[TextClassificationModel](https://github.com/NVIDIA/NeMo/blob/stable/nemo/collections/nlp/models/text_classification/text_classification_model.py) in NeMo supports text classification problems such as sentiment analysis or domain/intent detection for dialogue systems, as long as the data follows the format specified below. 

TextClassificationModel requires the data to be stored in TAB separated files (.tsv) with two columns of sentence and label. Each line of the data file contains text sequences, where words are separated with spaces and label separated with [TAB], i.e.: 

```
[WORD][SPACE][WORD][SPACE][WORD][TAB][LABEL]
```

For example:
```
hide new secretions from the parental units[TAB]0

that loves its characters and communicates something rather beautiful about human nature[TAB]1
...
```


If your dataset is stored in another format, you need to convert it to this format to use the TextClassificationModel. 

## Preprocess the Data

First, you need to download the zipped file of the SST-2 dataset from the GLUE Benchmark website: https://gluebenchmark.com/tasks, and put it in the current folder. Then the following script would extract it into the data path specified by `DATA_DIR`:

In [3]:
DATA_DIR = "Data"
WORK_DIR = "Work"

Now, the data folder should contain the following files:



* train.tsv
* dev.tsv
* test.tsv


The format of `train.tsv` and `dev.tsv` is close to NeMo's format except to have an extra header line at the beginning of the files. We would remove these extra lines. But `test.tsv` has different format and labels are missing for this part of the data.


# Model Configuration

Now, let's take a closer look at the model's configuration and learn to train the model from scratch and finetune the pretrained model.

Our text classification model uses a pretrained [BERT](https://arxiv.org/pdf/1810.04805.pdf) model (or other BERT-like models) followed by a classification layer on the output of the first token ([CLS]).

The model is defined in a config file which declares multiple important sections. The most important ones are:
- **model**: All arguments that are related to the Model - language model, tokenizer, head classifier, optimizer, schedulers, and datasets/data loaders.

- **trainer**: Any argument to be passed to PyTorch Lightning including number of epochs, number of GPUs, precision level, etc.


In [4]:
# download the model's configuration file 
MODEL_CONFIG = "text_classification_config.yaml"
CONFIG_DIR = WORK_DIR + '/configs/'

os.makedirs(CONFIG_DIR, exist_ok=True)
if not os.path.exists(CONFIG_DIR + MODEL_CONFIG):
    print('Downloading config file...')
    wget.download(f'https://raw.githubusercontent.com/NVIDIA/NeMo/{BRANCH}/examples/nlp/text_classification/conf/' + MODEL_CONFIG, CONFIG_DIR)
    print('Config file downloaded!')
else:
    print ('config file already exists')
config_path = f'{WORK_DIR}/configs/{MODEL_CONFIG}'
print(config_path)
config = OmegaConf.load(config_path)

config file already exists
2-jamoa uchun/Work/configs/text_classification_config.yaml


# Model Training From Scratch
## Setting up data within the config

We first need to set the num_classes in the config file which specifies the number of classes in the dataset. For SST-2, we have just two classes (0-positive and 1-negative). So we set the num_classes to 2. The model supports more than 2 classes too.



In [5]:
config.model.dataset.num_classes=9


Among other things, the config file contains dictionaries called dataset, train_ds and validation_ds. These are configurations used to setup the Dataset and DataLoaders of the corresponding config.

Notice that some config lines, including `model.dataset.classes_num`, have `???` as their value, this means that values for these fields are required to be to be specified by the user. We need to specify and set the `model.train_ds.file_name`, `model.validation_ds.file_name`, and `model.test_ds.file_name` in the config file to the paths of the train, validation, and test files if they exist. We may do it by updating the config file or by setting them from the command line. 

Let's now set the train and validation paths in the config.

In [6]:
config.model.train_ds.file_path = os.path.join(DATA_DIR, 'train.txt')
config.model.validation_ds.file_path = os.path.join(DATA_DIR, 'test.txt')
# Name of the .nemo file where trained model will be saved.
config.save_to = 'trained-model.nemo'
config.export_to = 'trained-model.onnx'
config.model.train_ds.batch_size = 8

print("Train dataloader's config: \n")
# OmegaConf.to_yaml() is used to create a proper format for printing the train dataloader's config
# You may change other params like batch size or the number of samples to be considered (-1 means all the samples)
print(OmegaConf.to_yaml(config.model.train_ds))

Train dataloader's config: 

file_path: 2-jamoa uchun/Data/train.txt
batch_size: 8
shuffle: true
num_samples: -1
num_workers: 3
drop_last: false
pin_memory: false



## Building the PyTorch Lightning Trainer

NeMo models are primarily PyTorch Lightning (PT) modules - and therefore are entirely compatible with the PyTorch Lightning ecosystem.

Let's first instantiate a PT Trainer object by using the trainer section of the config.

In [7]:
print("Trainer config - \n")
# OmegaConf.to_yaml() is used to create a proper format for printing the trainer config
print(OmegaConf.to_yaml(config.trainer))

Trainer config - 

devices: 1
num_nodes: 1
max_epochs: 100
max_steps: -1
accumulate_grad_batches: 1
gradient_clip_val: 0.0
precision: 32
accelerator: gpu
log_every_n_steps: 1
val_check_interval: 1.0
resume_from_checkpoint: null
num_sanity_val_steps: 0
enable_checkpointing: false
logger: false



First you need to create a PT trainer with the params stored in the trainer's config. You may set the number of steps for training with max_steps or number of epochs with max_epochs in the trainer's config.

In [8]:
# lets modify some trainer configs
# checks if we have GPU available and uses it
config.trainer.accelerator = 'gpu' if torch.cuda.is_available() else 'cpu'
config.trainer.devices = 1


# for mixed precision training, uncomment the lines below (precision should be set to 16 and amp_level to O1):
# config.trainer.precision = 16
# config.trainer.amp_level = O1

# disable distributed training when using Colab to prevent the errors
config.trainer.strategy = None

# setup max number of steps to reduce training time for demonstration purposes of this tutorial
# Training stops when max_step or max_epochs is reached (earliest)
config.trainer.max_epochs = 10

# instantiates a PT Trainer object by using the trainer section of the config
trainer = pl.Trainer(**config.trainer)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
`Trainer(val_check_interval=1.0)` was configured so validation will run at the end of the training epoch..


## Setting up the NeMo Experiment¶


NeMo has an experiment manager that handles the logging and saving checkpoints for us, so let's setup it. We need the PT trainer and the exp_manager config:

In [9]:
# The experiment manager of a trainer object can not be set twice. We repeat the trainer creation code again here to prevent getting error when this cell is executed more than once. 
trainer = pl.Trainer(**config.trainer)

# exp_dir specifies the path to store the the checkpoints and also the logs, it's default is "./nemo_experiments"
# You may set it by uncommentig the following line
# config.exp_manager.exp_dir = 'LOG_CHECKPOINT_DIR'

# OmegaConf.to_yaml() is used to create a proper format for printing the trainer config
print(OmegaConf.to_yaml(config.exp_manager))

exp_dir = exp_manager(trainer, config.exp_manager)

# the exp_dir provides a path to the current experiment for easy access
print(exp_dir)


GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
`Trainer(val_check_interval=1.0)` was configured so validation will run at the end of the training epoch..


exp_dir: null
name: TextClassification
create_tensorboard_logger: true
create_checkpoint_callback: true

[NeMo I 2023-01-28 22:27:15 exp_manager:343] Experiments will be logged at /home/airi/Desktop/jupyter/til model/text classification/nemo_experiments/TextClassification/2023-01-28_22-27-15
[NeMo I 2023-01-28 22:27:15 exp_manager:718] TensorboardLogger has been set up


[NeMo W 2023-01-28 22:27:15 exp_manager:988] The checkpoint callback was told to monitor a validation value and trainer's max_steps was set to -1. Please ensure that max_steps will run for at least 1 epochs to ensure that checkpointing will not error out.


/home/airi/Desktop/jupyter/til model/text classification/nemo_experiments/TextClassification/2023-01-28_22-27-15


Before initializing the model, we might want to modify some of the model configs. For example, we might want to modify the pretrained BERT model to another model. The default model is `bert-base-uncased`. We support a variety of models including all the models available in HuggingFace, and Megatron.

In [10]:
# complete list of supported BERT-like models
print(nemo_nlp.modules.get_pretrained_lm_models_list())

['bert-base-uncased', 'bert-large-uncased', 'bert-base-cased', 'bert-large-cased', 'bert-base-multilingual-uncased', 'bert-base-multilingual-cased', 'bert-base-chinese', 'bert-base-german-cased', 'bert-large-uncased-whole-word-masking', 'bert-large-cased-whole-word-masking', 'bert-large-uncased-whole-word-masking-finetuned-squad', 'bert-large-cased-whole-word-masking-finetuned-squad', 'bert-base-cased-finetuned-mrpc', 'bert-base-german-dbmdz-cased', 'bert-base-german-dbmdz-uncased', 'cl-tohoku/bert-base-japanese', 'cl-tohoku/bert-base-japanese-whole-word-masking', 'cl-tohoku/bert-base-japanese-char', 'cl-tohoku/bert-base-japanese-char-whole-word-masking', 'TurkuNLP/bert-base-finnish-cased-v1', 'TurkuNLP/bert-base-finnish-uncased-v1', 'wietsedv/bert-base-dutch-cased', 'distilbert-base-uncased', 'distilbert-base-uncased-distilled-squad', 'distilbert-base-cased', 'distilbert-base-cased-distilled-squad', 'distilbert-base-german-cased', 'distilbert-base-multilingual-cased', 'distilbert-base

In [11]:
# specify the BERT-like model, you want to use
# set the `model.language_modelpretrained_model_name' parameter in the config to the model you want to use
config.model.language_model.pretrained_model_name = "bert-base-uncased"

Now, we are ready to initialize our model. During the model initialization call, the dataset and data loaders will also be prepared for the training and validation.

Also, the pretrained BERT model will be automatically downloaded. Note it can take up to a few minutes depending on the size of the chosen BERT model for the first time you create the model. If your dataset is large, it also may take some time to read and process all the datasets. 

Now we can create the model with the model config and the trainer object like this:

In [12]:
model = nemo_nlp.models.TextClassificationModel(cfg=config.model, trainer=trainer)

[NeMo I 2023-01-28 22:27:27 tokenizer_utils:130] Getting HuggingFace AutoTokenizer with pretrained_model_name: bert-base-uncased, vocab_file: None, merges_files: None, special_tokens_dict: {}, and use_fast: False


Using eos_token, but it is not set yet.
Using bos_token, but it is not set yet.
[NeMo W 2023-01-28 22:27:32 modelPT:222] You tried to register an artifact under config key=tokenizer.vocab_file but an artifact for it has already been registered.


[NeMo I 2023-01-28 22:27:32 text_classification_dataset:120] Read 962 examples from 2-jamoa uchun/Data/train.txt.
[NeMo I 2023-01-28 22:27:32 text_classification_dataset:238] *** Example ***
[NeMo I 2023-01-28 22:27:32 text_classification_dataset:239] example 0: ['ekologiya', 'atrof', 'muhitni', 'muhofaza', 'qilish', 'tabiiy', 'resurslardan', 'oqilona', 'foydalanish', 'ularni', 'qayta', 'tiklash', 'va', 'inson', 'faoliyatining', 'tabiatga', 'salbiy', 'taʼsiri', 'oldini', 'olish', 'sohasida', 'yagona', 'davlat', 'siyosatini', 'amalga', 'oshirish', 'zarur']
[NeMo I 2023-01-28 22:27:32 text_classification_dataset:240] subtokens: [CLS] ek ##olo ##gi ##ya at ##ro ##f mu ##hit ##ni mu ##hof ##az ##a qi ##lish tab ##ii ##y res ##urs ##lard ##an o ##qi ##lon ##a f ##oy ##dal ##ani ##sh ul ##ar ##ni q ##ay ##ta ti ##kla ##sh va ins ##on fa ##oli ##yat ##ining tab ##ia ##t ##ga sal ##bi ##y ta ##ʼ ##sir ##i old ##ini ol ##ish so ##has ##ida ya ##gon ##a da ##v ##lat si ##yo ##sat ##ini ama ##lga

[NeMo W 2023-01-28 22:27:37 text_classification_dataset:250] Found 468 out of 962 sentences with more than 256 subtokens. Truncated long sentences from the end.


[NeMo I 2023-01-28 22:27:37 data_preprocessing:404] Some stats of the lengths of the sequences:
[NeMo I 2023-01-28 22:27:37 data_preprocessing:406] Min: 33 |                  Max: 256 |                  Mean: 185.73076923076923 |                  Median: 246.0
[NeMo I 2023-01-28 22:27:37 data_preprocessing:412] 75 percentile: 256.00
[NeMo I 2023-01-28 22:27:37 data_preprocessing:413] 99 percentile: 256.00
[NeMo I 2023-01-28 22:27:37 text_classification_dataset:120] Read 9 examples from 2-jamoa uchun/Data/test.txt.
[NeMo I 2023-01-28 22:27:37 text_classification_dataset:238] *** Example ***
[NeMo I 2023-01-28 22:27:37 text_classification_dataset:239] example 0: ['bugungi', 'kunda', 'mamlakatimizda', 'chuqur', 'demokratik', 'islohotlar', 'amalga', 'oshirilmoqda', 'demokratiyaning', 'asosiy', 'elementlaridan', 'biri', 'bu', 'ommaviy', 'axborot', 'vositalari', 'erkinligi', 'va', 'ular', 'huquqlarining', 'qonun', 'bilan', 'kafolatlanganligi', 'hisoblanadi', 'demokratik', 'islohotlarni', 'ya

[NeMo W 2023-01-28 22:27:37 text_classification_dataset:250] Found 9 out of 9 sentences with more than 256 subtokens. Truncated long sentences from the end.


[NeMo I 2023-01-28 22:27:37 data_preprocessing:404] Some stats of the lengths of the sequences:
[NeMo I 2023-01-28 22:27:37 data_preprocessing:406] Min: 256 |                  Max: 256 |                  Mean: 256.0 |                  Median: 256.0
[NeMo I 2023-01-28 22:27:37 data_preprocessing:412] 75 percentile: 256.00
[NeMo I 2023-01-28 22:27:37 data_preprocessing:413] 99 percentile: 256.00
[NeMo I 2023-01-28 22:27:37 text_classification_model:193] Dataloader config or file_path for the test is missing, so no data loader for test is created!


[NeMo W 2023-01-28 22:27:37 nlp_overrides:229] Apex was not found. Please see the NeMo README for installation instructions: https://github.com/NVIDIA/apex
    Megatron-based models require Apex to function correctly.
Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceCla

## Training

You may start the training by using the trainer.fit() method. The number of steps/epochs of the training are specified already in the config of the trainer and you may update them before creating the trainer.

In [13]:
# start model training
trainer.fit(model)
model.save_to(config.save_to)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1]


[NeMo I 2023-01-28 22:28:32 modelPT:602] Optimizer config = Adam (
    Parameter Group 0
        amsgrad: False
        betas: [0.9, 0.999]
        capturable: False
        differentiable: False
        eps: 1e-08
        foreach: None
        fused: False
        lr: 2e-05
        maximize: False
        weight_decay: 0.01
    )
[NeMo I 2023-01-28 22:28:32 lr_scheduler:910] Scheduler "<nemo.core.optim.lr_scheduler.WarmupAnnealing object at 0x7f0fe42ff640>" 
    will be used during training (effective maximum steps = 1210) - 
    Parameters : 
    (warmup_steps: null
    warmup_ratio: 0.1
    last_epoch: -1
    max_steps: 1210
    )



  | Name                  | Type                 | Params
---------------------------------------------------------------
0 | loss                  | CrossEntropyLoss     | 0     
1 | bert_model            | BertEncoder          | 109 M 
2 | classifier            | SequenceClassifier   | 597 K 
3 | classification_report | ClassificationReport | 0     
---------------------------------------------------------------
110 M     Trainable params
0         Non-trainable params
110 M     Total params
440.319   Total estimated model params size (MB)


Training: 0it [00:00, ?it/s]

[NeMo W 2023-01-28 22:28:32 nemo_logging:349] /home/airi/anaconda3/lib/python3.9/site-packages/nemo/collections/nlp/data/text_classification/text_classification_dataset.py:203: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:230.)
      torch.LongTensor(padded_input_ids),
    
[NeMo W 2023-01-28 22:28:32 nemo_logging:349] /home/airi/anaconda3/lib/python3.9/site-packages/nemo/collections/nlp/data/text_classification/text_classification_dataset.py:203: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:230.)
      torch.LongTensor(padded_input_ids),
    
[NeMo W 2023-01-28 22:28:32 nemo_log

...

Epoch 8, global step 1089: 'val_loss' reached 1.64826 (best 1.64826), saving model to '/home/airi/Desktop/jupyter/til model/text classification/nemo_experiments/TextClassification/2023-01-28_22-27-15/checkpoints/TextClassification--val_loss=1.6483-epoch=8.ckpt' as top 3
[NeMo W 2023-01-28 22:33:22 nemo_logging:349] /home/airi/anaconda3/lib/python3.9/site-packages/nemo/collections/nlp/data/text_classification/text_classification_dataset.py:203: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:230.)
      torch.LongTensor(padded_input_ids),
    
[NeMo W 2023-01-28 22:33:22 nemo_logging:349] /home/airi/anaconda3/lib/python3.9/site-packages/nemo/collections/nlp/data/text_classification/text_classification_dataset.py:203: UserWarning: Creating a tensor from a list of numpy.ndarrays is extrem

Validation: 0it [00:00, ?it/s]

[NeMo W 2023-01-28 22:33:51 nemo_logging:349] /home/airi/anaconda3/lib/python3.9/site-packages/nemo/collections/nlp/data/text_classification/text_classification_dataset.py:203: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:230.)
      torch.LongTensor(padded_input_ids),
    


[NeMo I 2023-01-28 22:33:51 text_classification_model:142] val_report: 
    label                                                precision    recall       f1           support   
    label_id: 0                                              0.00       0.00       0.00          1
    label_id: 1                                              0.00       0.00       0.00          1
    label_id: 2                                              0.00       0.00       0.00          1
    label_id: 3                                              0.00       0.00       0.00          1
    label_id: 4                                              0.00       0.00       0.00          1
    label_id: 5                                             50.00     100.00      66.67          1
    label_id: 6                                            100.00     100.00     100.00          1
    label_id: 7                                            100.00     100.00     100.00          1
    label_id: 8              

Epoch 9, global step 1210: 'val_loss' reached 1.58924 (best 1.58924), saving model to '/home/airi/Desktop/jupyter/til model/text classification/nemo_experiments/TextClassification/2023-01-28_22-27-15/checkpoints/TextClassification--val_loss=1.5892-epoch=9.ckpt' as top 3
`Trainer.fit` stopped: `max_epochs=10` reached.


# Evaluation

To see how the model performs, we can run evaluate and test the performance of the trained model on a data file. Here we would load the best checkpoint (the one with the lowest validation loss) and create a model (eval_model) from the checkpoint. We would also create a new trainer (eval_trainer) to show how it is done when training is done and you have just the checkpoints. If you want to perform the evaluation in the same script as the training's script, you may still use the same model and trainer you used for training.

In [14]:
# extract the path of the best checkpoint from the training, you may update it to any checkpoint
checkpoint_path = trainer.checkpoint_callback.best_model_path
# Create an evaluation model and load the checkpoint
eval_model = nemo_nlp.models.TextClassificationModel.load_from_checkpoint(checkpoint_path=checkpoint_path)

# create a dataloader config for evaluation, the same data file provided in validation_ds is used here
# file_path can get updated with any file
eval_config = OmegaConf.create({'file_path': config.model.validation_ds.file_path, 'batch_size': 64, 'shuffle': False, 'num_samples': -1})
eval_model.setup_test_data(test_data_config=eval_config)
#eval_dataloader = eval_model._create_dataloader_from_config(cfg=eval_config, mode='test')

# a new trainer is created to show how to evaluate a checkpoint from an already trained model
# create a copy of the trainer config and update it to be used for final evaluation
eval_trainer_cfg = config.trainer.copy()
eval_trainer_cfg.accelerator = 'gpu' if torch.cuda.is_available() else 'cpu' # it is safer to perform evaluation on single GPU as PT is buggy with the last batch on multi-GPUs
eval_trainer_cfg.strategy = None # 'ddp' is buggy with test process in the current PT, it looks like it has been fixed in the latest master
eval_trainer = pl.Trainer(**eval_trainer_cfg)
eval_trainer.test(model=eval_model, verbose=False) # test_dataloaders=eval_dataloader,

[NeMo W 2023-01-28 22:50:17 nemo_logging:349] /home/airi/anaconda3/lib/python3.9/site-packages/pytorch_lightning/utilities/cloud_io.py:41: LightningDeprecationWarning: `pytorch_lightning.utilities.cloud_io.load` has been deprecated in v1.8.0 and will be removed in v1.10.0. This function is internal but you can copy over its implementation.
      rank_zero_deprecation(
    


[NeMo I 2023-01-28 22:50:17 tokenizer_utils:130] Getting HuggingFace AutoTokenizer with pretrained_model_name: bert-base-uncased, vocab_file: /home/airi/.cache/huggingface/nemo_nlp_tmp/ae0d012864bdb2474ba67537c3f5e0fa/vocab.txt, merges_files: None, special_tokens_dict: {}, and use_fast: False


Using eos_token, but it is not set yet.
Using bos_token, but it is not set yet.
[NeMo W 2023-01-28 22:50:23 modelPT:222] You tried to register an artifact under config key=tokenizer.vocab_file but an artifact for it has already been registered.
[NeMo W 2023-01-28 22:50:23 modelPT:142] If you intend to do training or fine-tuning, please call the ModelPT.setup_training_data() method and provide a valid configuration file to setup the train data loader.
    Train config : 
    file_path: 2-jamoa uchun/Data/train.txt
    batch_size: 8
    shuffle: true
    num_samples: -1
    num_workers: 3
    drop_last: false
    pin_memory: false
    
[NeMo W 2023-01-28 22:50:23 modelPT:149] If you intend to do validation, please call the ModelPT.setup_validation_data() or ModelPT.setup_multiple_validation_data() method and provide a valid configuration file to setup the validation data loader(s). 
    Validation config : 
    file_path: 2-jamoa uchun/Data/test.txt
    batch_size: 64
    shuffle: false


[NeMo I 2023-01-28 22:50:27 text_classification_dataset:120] Read 9 examples from 2-jamoa uchun/Data/test.txt.
[NeMo I 2023-01-28 22:50:27 text_classification_dataset:238] *** Example ***
[NeMo I 2023-01-28 22:50:27 text_classification_dataset:239] example 0: ['bugungi', 'kunda', 'mamlakatimizda', 'chuqur', 'demokratik', 'islohotlar', 'amalga', 'oshirilmoqda', 'demokratiyaning', 'asosiy', 'elementlaridan', 'biri', 'bu', 'ommaviy', 'axborot', 'vositalari', 'erkinligi', 'va', 'ular', 'huquqlarining', 'qonun', 'bilan', 'kafolatlanganligi', 'hisoblanadi', 'demokratik', 'islohotlarni', 'yana', 'da', 'mustahkamlash', 'maqsadida', 'ommaviy', 'axborot', 'vositalari', 'xodimlarining', 'qonuniy', 'faoliyatiga', 'aralashganlik', 'va', 'toʻsqinlik', 'qilganlik', 'uchun', 'maʼmuriy', 'va', 'jinoiy', 'javobgarlik', 'joriy', 'qilishni', 'taklif', 'qilamiz', 'buning', 'uchun', 'oʻzbekiston', 'respublikasi', 'maʼmuriy', 'javobgarlik', 'toʻgʻrisidagi', 'va', 'jinoyat', 'kodekslariga', 'oʻzgartish', 'va'

[NeMo W 2023-01-28 22:50:27 text_classification_dataset:250] Found 9 out of 9 sentences with more than 256 subtokens. Truncated long sentences from the end.


...

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
`Trainer(val_check_interval=1.0)` was configured so validation will run at the end of the training epoch..
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1]
[NeMo W 2023-01-28 22:50:27 nemo_logging:349] /home/airi/anaconda3/lib/python3.9/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:224: PossibleUserWarning: The dataloader, test_dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 64 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
      rank_zero_warn(
    


Testing: 0it [00:00, ?it/s]

[NeMo W 2023-01-28 22:50:27 nemo_logging:349] /home/airi/anaconda3/lib/python3.9/site-packages/nemo/collections/nlp/data/text_classification/text_classification_dataset.py:203: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:230.)
      torch.LongTensor(padded_input_ids),
    


[NeMo I 2023-01-28 22:50:27 text_classification_model:142] test_report: 
    label                                                precision    recall       f1           support   
    label_id: 0                                              0.00       0.00       0.00          1
    label_id: 1                                              0.00       0.00       0.00          1
    label_id: 2                                              0.00       0.00       0.00          1
    label_id: 3                                              0.00       0.00       0.00          1
    label_id: 4                                              0.00       0.00       0.00          1
    label_id: 5                                             50.00     100.00      66.67          1
    label_id: 6                                            100.00     100.00     100.00          1
    label_id: 7                                            100.00     100.00     100.00          1
    label_id: 8             

[{'test_loss': 1.5892386436462402,
  'test_precision': 44.44443893432617,
  'test_f1': 44.44443893432617,
  'test_recall': 44.44443893432617}]